In [10]:
import ast
import pickle

import mlflow
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from graph_reinforcement_learning_using_blockchain_data import config

config.load_dotenv()
mlflow.set_tracking_uri(uri=config.MLFLOW_TRACKING_URI)

In [11]:
df_class0 = pd.read_csv(config.RAW_DATA_DIR / "receipts_class0.csv")
df_class1 = pd.read_csv(config.RAW_DATA_DIR / "receipts_class1.csv")
df_eth_balances_class1 = pd.read_csv(config.RAW_DATA_DIR / "eth_balances_class1.csv")
df_eth_balances_class0 = pd.read_csv(config.RAW_DATA_DIR / "eth_balances_class0.csv")

with open(config.FLASHBOTS_Q2_DATA_DIR / "train_accounts.pkl", "rb") as f:
    train_accounts = pickle.load(f)

with open(config.FLASHBOTS_Q2_DATA_DIR / "test_accounts.pkl", "rb") as f:
    test_accounts = pickle.load(f)

In [12]:
df_class0_with_eth_balances = df_class0.merge(
    df_eth_balances_class0,
    left_on=["from", "blockNumber"],
    right_on=["account", "block_number"],
    how="inner",
)
df_class1_with_eth_balances = df_class1.merge(
    df_eth_balances_class1,
    left_on=["from", "blockNumber"],
    right_on=["account", "block_number"],
    how="inner",
)

In [13]:
df_class0_multi_occ = df_class0_with_eth_balances[
    df_class0_with_eth_balances["from"].duplicated(keep=False)
]

In [14]:
df_class0_with_eth_balances["label"] = 0
df_class1_with_eth_balances["label"] = 1

In [15]:
df_receipts = pd.concat(
    [df_class0_with_eth_balances, df_class1_with_eth_balances], ignore_index=True
)
df_receipts.drop_duplicates("transaction_hash", inplace=True)

In [16]:
df_median_gas_prices = pd.DataFrame(
    {
        "median_gas_price": df_receipts.groupby(["blockNumber"])["effectiveGasPrice"].median(),
        "std_gas_price": df_receipts.groupby(["blockNumber"])["effectiveGasPrice"].std(),
        "max_gas_price": df_receipts.groupby(["blockNumber"])["effectiveGasPrice"].max(),
        "min_gas_price": df_receipts.groupby(["blockNumber"])["effectiveGasPrice"].min(),
    }
)

df_with_median_gas_prices = df_receipts.merge(df_median_gas_prices, how="left", on="blockNumber")
df_with_median_gas_prices.head()

,block_number_x,transaction_hash,blockHash,blockNumber,logsBloom,gasUsed,contractAddress,cumulativeGasUsed,transactionIndex,from,...,profit_token_address,start_amount,end_amount,profit_amount,error,protocols,median_gas_price,std_gas_price,max_gas_price,min_gas_price
0,16950601,0x4d419ef95a879f7d3da5ad7b55289ba88e360d5df7d4...,0x196c250ac3f953b996f9024ec93c3cd073c5f31fc150...,16950601,0x00000000000000000000000000000000000000000000...,21000,NaN,22771101,327,0x18dE03D26266877f316A3b2c89383cC9f54777b6,...,NaN,NaN,NaN,NaN,NaN,NaN,1.829850e+10,6.664875e+09,50000000000,18000000000
1,16950601,0x5f7a16727b0ed026b1656924be454df15951fc3d1350...,0x196c250ac3f953b996f9024ec93c3cd073c5f31fc150...,16950601,0x00000000000000000000000000000000000000000000...,21000,NaN,4567366,57,0x8216874887415e2650D12D53Ff53516F04a74FD7,...,NaN,NaN,NaN,NaN,NaN,NaN,1.829850e+10,6.664875e+09,50000000000,18000000000
2,16950601,0x07d24a0c6b2837c5731ad659f62c59b935b2195e70dc...,0x196c250ac3f953b996f9024ec93c3cd073c5f31fc150...,16950601,0x00200000000000000000800080000000000800000000...,157316,NaN,2135061,12,0xf44329d7915C95Cad0510D8174accDFBceABc16d,...,NaN,NaN,NaN,NaN,NaN,NaN,1.829850e+10,6.664875e+09,50000000000,18000000000
3,16950601,0x93211130eb153a9aafe8f6369f747e894750723a51c2...,0x196c250ac3f953b996f9024ec93c3cd073c5f31fc150...,16950601,0x00000000000000000000000000000000000000000000...,21000,NaN,25771997,379,0x8B820E319cbdbd94E77751Aa1A6C92B429eAd1C9,...,NaN,NaN,NaN,NaN,NaN,NaN,1.829850e+10,6.664875e+09,50000000000,18000000000
4,16950601,0x7e3b9d96c2c7b2e3fd4cf4fb8b8a97f004861d6087fc...,0x196c250ac3f953b996f9024ec93c3cd073c5f31fc150...,16950601,0x00000000000000000000000000000000000000000000...,48537,NaN,10131517,140,0x4E5B2e1dc63F6b91cb6Cd759936495434C7e972F,...,NaN,NaN,NaN,NaN,NaN,NaN,1.829850e+10,6.664875e+09,50000000000,18000000000


In [17]:
df_with_actions = df_with_median_gas_prices.copy()
df_with_actions["action"] = df_with_median_gas_prices.apply(
    lambda r: 1 if r["effectiveGasPrice"] > r["median_gas_price"] else 0, axis=1
)

In [18]:
df_with_actions["action"].mean()

0.3064463690632084

In [19]:
df_with_actions.rename(columns={"balance": "eth_balance"}, inplace=True)
df_with_actions["eth_balance"] = df_with_actions["eth_balance"].astype("float64")
df_with_actions["median_gas_price"] = df_with_actions["median_gas_price"].astype("float64")
df_with_actions["std_gas_price"] = df_with_actions["std_gas_price"].astype("float64")
df_with_actions["from"] = df_with_actions["from"].astype("string")
df_with_actions["to"] = df_with_actions["to"].astype("string")
df_with_actions.head()

,block_number_x,transaction_hash,blockHash,blockNumber,logsBloom,gasUsed,contractAddress,cumulativeGasUsed,transactionIndex,from,...,start_amount,end_amount,profit_amount,error,protocols,median_gas_price,std_gas_price,max_gas_price,min_gas_price,action
0,16950601,0x4d419ef95a879f7d3da5ad7b55289ba88e360d5df7d4...,0x196c250ac3f953b996f9024ec93c3cd073c5f31fc150...,16950601,0x00000000000000000000000000000000000000000000...,21000,NaN,22771101,327,0x18dE03D26266877f316A3b2c89383cC9f54777b6,...,NaN,NaN,NaN,NaN,NaN,1.829850e+10,6.664875e+09,50000000000,18000000000,0
1,16950601,0x5f7a16727b0ed026b1656924be454df15951fc3d1350...,0x196c250ac3f953b996f9024ec93c3cd073c5f31fc150...,16950601,0x00000000000000000000000000000000000000000000...,21000,NaN,4567366,57,0x8216874887415e2650D12D53Ff53516F04a74FD7,...,NaN,NaN,NaN,NaN,NaN,1.829850e+10,6.664875e+09,50000000000,18000000000,1
2,16950601,0x07d24a0c6b2837c5731ad659f62c59b935b2195e70dc...,0x196c250ac3f953b996f9024ec93c3cd073c5f31fc150...,16950601,0x00200000000000000000800080000000000800000000...,157316,NaN,2135061,12,0xf44329d7915C95Cad0510D8174accDFBceABc16d,...,NaN,NaN,NaN,NaN,NaN,1.829850e+10,6.664875e+09,50000000000,18000000000,1
3,16950601,0x93211130eb153a9aafe8f6369f747e894750723a51c2...,0x196c250ac3f953b996f9024ec93c3cd073c5f31fc150...,16950601,0x00000000000000000000000000000000000000000000...,21000,NaN,25771997,379,0x8B820E319cbdbd94E77751Aa1A6C92B429eAd1C9,...,NaN,NaN,NaN,NaN,NaN,1.829850e+10,6.664875e+09,50000000000,18000000000,0
4,16950601,0x7e3b9d96c2c7b2e3fd4cf4fb8b8a97f004861d6087fc...,0x196c250ac3f953b996f9024ec93c3cd073c5f31fc150...,16950601,0x00000000000000000000000000000000000000000000...,48537,NaN,10131517,140,0x4E5B2e1dc63F6b91cb6Cd759936495434C7e972F,...,NaN,NaN,NaN,NaN,NaN,1.829850e+10,6.664875e+09,50000000000,18000000000,1


In [20]:
df_with_actions[df_with_actions["std_gas_price"].isna() == True]

,block_number_x,transaction_hash,blockHash,blockNumber,logsBloom,gasUsed,contractAddress,cumulativeGasUsed,transactionIndex,from,...,start_amount,end_amount,profit_amount,error,protocols,median_gas_price,std_gas_price,max_gas_price,min_gas_price,action
117655,16968233,0x63ad9854841203b4f389b5ba4d75a3c6dbfcfe518ad5...,0x5c54db5995cd5b1d1d31f08a0bd92a85cb3884e23d30...,16968233,0x00000000000000000000400000000000000000000000...,163204,NaN,377338,4,0x8874c1F934B5687FDE94594ceCa0C10a8a1500E0,...,103424647109058544,118151458169933656,14726811060875112,NaN,"[""uniswap_v3""]",2.332962e+10,NaN,23329619051,23329619051,0
132832,17067621,0x90f591b8c8192ec209057069734451522394f1d9aa99...,0xff9bd8c419d2f6bd7f045b0a3fee3717ec547f5adaa6...,17067621,0x00200000000000000000000080000000000004000000...,134884,NaN,664159,4,0x000000a010FBFEA4c98cff39C9CcC6f7A9D70906,...,219184736000000000,233443319727767297,14258583727767297,NaN,"[""uniswap_v2""]",1.049890e+11,NaN,104988960885,104988960885,0
132833,17038537,0xef668a185344db643e8ee0eab81fa677a6e66ae3434e...,0x51f901b3d2e7a0b956f6629a06e22bc0e6d0f2cd68f1...,17038537,0x00200000000800000000000081200000000080000000...,190715,NaN,1362971,5,0x255ED4ae17EFf35923cfDe87ED6ce81c4ebf8888,...,1391499728122342793,1405780788518086387,14281060395743594,NaN,"[""uniswap_v2"",""uniswap_v3""]",7.375846e+10,NaN,73758458903,73758458903,0
132834,17038535,0xbdcdc5a52680962d6f9d171e76aab5c06ccd2452f8bc...,0x89355ba23591e64efb188fb21a0c64769762ef39ed5d...,17038535,0x00200000000000000000000080000000000000000000...,191101,NaN,719310,4,0x76F36d497b51e48A288f03b4C1d7461e92247d5e,...,94705192638476201,99678543756114460,4973351117638259,NaN,"[""uniswap_v2"",""uniswap_v3""]",2.568748e+10,NaN,25687479672,25687479672,0
132835,17273160,0xd8f5f21c064f6cfb8d0dcc384124f12b2771582d3b66...,0x85c3167d297b14e2e1b4dde496adfa06cf56a50d8301...,17273160,0x00200000008000000000000080000000000000000000...,118254,NaN,454764,5,0xae2Fc483527B8EF99EB5D9B44875F005ba1FaE13,...,134969562025164800,152199836945350656,17230274920185856,NaN,"[""uniswap_v2""]",1.442486e+11,NaN,144248589301,144248589301,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221451,17416498,0x6c62e7a2fdecbe48b9c6782670526acf0a9ac66b3c74...,0x94affb0ce02fb6db6a7a08f5e0c6d14642875957d488...,17416498,0x00200000000000000000000080002008000000000000...,165619,NaN,597938,4,0x3001f6F2187D875a1Bc24B10Fe9616Ebcaf4fB45,...,213389918183882752,219686821276155904,6296903092273152,NaN,"[""uniswap_v2"",""uniswap_v3""]",3.767891e+10,NaN,37678913658,37678913658,0
221452,17416517,0xf6724df8887e2826203dd6875076885efff3b7ebb50d...,0xdc2bf217f8862ab6483249ff5eb7bcc1b4668981e2ab...,17416517,0x00200000002000000000000084000000000000000000...,191920,NaN,832689,4,0x76F36d497b51e48A288f03b4C1d7461e92247d5e,...,504498660940168064,516390509312555584,11891848372387520,NaN,"[""uniswap_v2"",""uniswap_v3""]",6.134290e+10,NaN,61342902712,61342902712,0
221453,17416528,0x2108b55bcb8c136213ee5a8af2b0603fad0b342dc80f...,0xe7a679ad77f6269c497102bce8a31c7d916dca19264f...,17416528,0x00a00000000000000000000080000001000000000000...,134876,NaN,1292403,10,0x3001f6F2187D875a1Bc24B10Fe9616Ebcaf4fB45,...,280398554827063296,285777365710143488,5378810883080192,NaN,"[""uniswap_v2"",""uniswap_v3""]",3.952164e+10,NaN,39521635799,39521635799,0
221454,17416532,0xfc1c3b09f893b987229da91093bc3e968e78715f8f67...,0xf42ac3d1822824a0786334d15872e7059cf67a4f9e39...,17416532,0x00200000008000000000000080200000000000000000...,252026,NaN,1875322,10,0xB543F8D66765c7074d1d77147d7e07880c96908F,...,67833499872104753,72568668813477947,4735168941373194,NaN,"[""uniswap_v2""]",3.536379e+10,NaN,35363787811,35363787811,0


In [21]:
df_with_actions = df_with_actions.fillna({"std_gas_price": 0})

In [22]:
df_with_actions_0 = df_with_actions[df_with_actions["label"] == 0]
df_with_actions_1 = df_with_actions[df_with_actions["label"] == 1]

In [23]:
validation_accounts, test_accounts_excl_val = train_test_split(
    test_accounts, test_size=0.5, random_state=42
)

In [24]:
df_with_actions_0_train = df_with_actions_0[df_with_actions_0["from"].isin(train_accounts)]
df_with_actions_0_val = df_with_actions_0[df_with_actions_0["from"].isin(validation_accounts)]
df_with_actions_0_test = df_with_actions_0[df_with_actions_0["from"].isin(test_accounts_excl_val)]

In [25]:
df_with_actions_1_train = df_with_actions_1[df_with_actions_1["from"].isin(train_accounts)]
df_with_actions_1_val = df_with_actions_1[df_with_actions_1["from"].isin(validation_accounts)]
df_with_actions_1_test = df_with_actions_1[df_with_actions_1["from"].isin(test_accounts_excl_val)]

In [26]:
df_val = pd.concat([df_with_actions_0_val, df_with_actions_1_val])

In [27]:
df_with_actions_0_train.to_csv(
    config.PROCESSED_DATA_DIR / "AIRL" / "airl_0_train.csv", index=False
)
df_with_actions_0_test.to_csv(config.PROCESSED_DATA_DIR / "AIRL" / "airl_0_test.csv", index=False)
df_with_actions_1_train.to_csv(
    config.PROCESSED_DATA_DIR / "AIRL" / "airl_1_train.csv", index=False
)
df_with_actions_1_test.to_csv(config.PROCESSED_DATA_DIR / "AIRL" / "airl_1_test.csv", index=False)
df_val.to_csv(config.PROCESSED_DATA_DIR / "AIRL" / "airl_val.csv", index=False)